## Importação da Biblioteca pandas para trabalhar os dados

In [52]:
import pandas as pd

## Definição de uma função para contar o número de acertos que o premiado teve

In [53]:
def cont_acertos(premiados_dezenas, sorteio_dezenas):
  return len(premiados_dezenas.intersection(sorteio_dezenas))

## Carregar os dados da base listapremiados.xlsx

In [54]:
df_premiados = pd.read_excel('listapremiados.xlsx', parse_dates=['premiados_mes'])

## As dezenas vêm como string todas na mesma célula separadas por espaço, aqui transformamos esta stringem um array

In [55]:
df_premiados['premiados_dezenas'] = df_premiados['premiados_dezenas'].str.split(' ')

## Verificando o tipo dos dados 

In [56]:
df_premiados.dtypes

premiados_mes                datetime64[ns]
premiados_cpf                        object
premiados_dezenas                    object
premiados_estabelecimento            object
premiados_municipio                  object
dtype: object

Carregamento dos dados dezenasSorteadas.xlsx

In [57]:
df_sorteios = pd.read_excel('dezenasSorteadas.xlsx', parse_dates=['sorteio_mes'])

## Verificando os tipos dos dados. O prinicpal é que sorteio_mes seja do tipo datetime pois será a chave de ligação
## entre df_premiados e df_sorteios

In [58]:
df_sorteios.dtypes

sorteio_mes        datetime64[ns]
sorteio_dezenas            object
dtype: object

## Assim como na lista de premiados aqui as dezenas também estão em uma string e sofre o mesmo tratamento

In [59]:
df_sorteios.sorteio_dezenas = df_sorteios.sorteio_dezenas.str.split(' ')

## Criando uma coluna mes em cada dataframe apenas para ser chave de ligação entre elas

In [60]:
df_sorteios['mes'] = df_sorteios['sorteio_mes']
df_premiados['mes'] = df_premiados['premiados_mes']


## Fazendo merge dos dataframes para poder ver os numeros que cada premiado acertou

In [61]:
df_premiados = pd.merge(df_premiados,df_sorteios,on='mes')

## Transofrmando cada valor de _dezenas em um set permitindo usar as funções de conjunto para facilitar a conferencia do sorteio

In [62]:
df_premiados['premiados_dezenas'] = df_premiados['premiados_dezenas'].apply(lambda x: set(x))
df_premiados['sorteio_dezenas'] = df_premiados['sorteio_dezenas'].apply(lambda x: set(x))

## Aplicando a função anteriormente definida -- cont_acertos -- para cada linha e criando a cooluna 'acertos' para armazenar o resultado

In [63]:
df_premiados['acertos'] = df_premiados.apply(lambda x: cont_acertos(x.premiados_dezenas, x.sorteio_dezenas), axis=1)

## Premiados por mês e cidadade
### Agrupando os valores por mês e cidade com a função agregadora 'count' sobre a coluna premiados_cpf
### Rearranjando o dataframe por meio da nlargest para que cada agrupamento respeite a ordem do valor em count

In [64]:
df_premiados_mes_municipio = df_premiados.groupby(['mes','premiados_municipio']).agg({'count'})['premiados_cpf']
df_premiados_mes_municipio = df_premiados_mes_municipio['count'].groupby('mes',group_keys=False).nlargest(999)
df_premiados_mes_municipio

mes         premiados_municipio 
2020-07-01  Campo Grande            142
            Dourados                 39
            Corumbá                  23
            Três Lagoas              20
            Nova Andradina           10
                                   ... 
2021-07-01  Bela Vista                1
            Batayporã                 1
            Aparecida do Taboado      1
            Anaurilândia              1
            Vicentina                 1
Name: count, Length: 608, dtype: int64

## Criando uma pivot table a partir de df_premiados_mes_municipio

In [65]:
df_premiados_mes_municipio_pivot = df_premiados
df_premiados_mes_municipio_pivot.fillna(0)
df_premiados_mes_municipio_pivot = pd.pivot_table(
  df_premiados_mes_municipio_pivot,
  index= ['premiados_municipio'],
  columns = ['mes'],
  margins=True,
  values='premiados_cpf',
  aggfunc='count',
  fill_value=0
)
df_premiados_mes_municipio_pivot.sort_values(by=('All'), ascending=False, inplace=True)
df_premiados_mes_municipio_pivot

mes,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,All
premiados_municipio,,,,,,,,,,,,,,
All,343,365,363,373,394,374,382,345,349,318,350,359,337,4652
Campo Grande,142,160,153,170,195,170,170,164,174,132,166,168,149,2113
Dourados,39,31,45,46,53,49,42,45,30,35,41,36,31,523
Três Lagoas,20,24,19,21,21,16,28,15,17,24,15,21,21,262
Corumbá,23,9,14,9,17,16,14,9,13,11,12,18,14,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jaraguari,0,0,1,0,0,0,0,1,0,0,0,0,0,2
Corguinho,0,0,0,0,0,0,0,1,0,0,0,0,0,1
Aral Moreira,0,0,0,0,0,1,0,0,0,0,0,0,0,1


## Criando uma pivot table para mostrar por mês e municpio a quantidade de ganhadores que acertaram as 6 dezenas

In [66]:
df_premiados.sort_values(['acertos'], ascending=False).where(df_premiados['acertos'] == 6)
df_ganhadores_6_acertos_mes_municipio = pd.pivot_table(
  df_premiados.sort_values(['acertos'], ascending=False).where(df_premiados['acertos'] == 6),
  index= ['premiados_municipio'],
  columns = ['mes'],
  margins=True,
  values='premiados_cpf',
  aggfunc='count',
  fill_value=0
)
df_ganhadores_6_acertos_mes_municipio

mes,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,All
premiados_municipio,,,,,,,,,,,,,,
Bela Vista,0,0,0,0,0,1,0,0,1,0,1,0,0,3
Campo Grande,1,1,1,0,0,4,1,1,1,2,0,1,1,14
Corumbá,0,0,0,0,1,0,0,0,0,0,0,0,0,1
Dourados,0,0,1,0,0,2,0,0,0,0,0,1,0,4
Maracaju,0,0,0,1,1,1,0,0,0,0,0,0,0,3
Mundo Novo,0,0,0,0,0,0,1,0,0,0,0,0,0,1
Naviraí,0,0,0,0,0,0,1,0,0,0,0,0,0,1
Nova Andradina,0,0,0,0,0,0,0,0,0,0,0,1,0,1
Sidrolândia,0,0,0,0,0,0,1,0,0,0,0,0,0,1


## Listando por mes e municipio os ganhadores que acertaram as 6 dezenas

In [67]:
df_premiados_6_acertos_cidade = df_premiados.where(df_premiados['acertos'] ==6)
df_premiados_6_acertos_cidade.dropna()
df_premiados_6_acertos_cidade = df_premiados_6_acertos_cidade.groupby(['mes','premiados_municipio']).agg({'count'})['premiados_cpf']
df_premiados_6_acertos_cidade = df_premiados_6_acertos_cidade['count'].groupby('mes',group_keys=False).nlargest(999)
df_premiados_6_acertos_cidade

mes         premiados_municipio 
2020-07-01  Campo Grande            1
2020-08-01  Campo Grande            1
2020-09-01  Campo Grande            1
            Dourados                1
2020-10-01  Maracaju                1
2020-11-01  Corumbá                 1
            Maracaju                1
            Três Lagoas             1
2020-12-01  Campo Grande            4
            Dourados                2
            Bela Vista              1
            Maracaju                1
2021-01-01  Campo Grande            1
            Mundo Novo              1
            Naviraí                 1
            Sidrolândia             1
            Três Lagoas             1
2021-02-01  Campo Grande            1
            Três Lagoas             1
2021-03-01  Bela Vista              1
            Campo Grande            1
2021-04-01  Campo Grande            2
2021-05-01  Bela Vista              1
            São Gabriel do Oeste    1
2021-06-01  Campo Grande            1
            Doura

## Listando por CPF e municipio aqueles que foram premiados mias de uma
### Como o número completo do CPF é sigiloso está sendo admitido que apenas o mesmo código de CPF em combinação com a mesma cidade é uma pessoa única
### Ignorando a possibilidade de o mesmo CPF ganhar em outra cidade, criando asism um novo resigtro

In [68]:
df_sortudos = df_premiados.groupby(['premiados_municipio','premiados_cpf']).agg({'count'})['premiados_dezenas']
df_sortudos = df_sortudos.where(df_sortudos['count'] >1 ).dropna()
df_sortudos

count
premiados_municipio premiados_cpf       
Campo Grande        ***029.931**     2.0
                    ***056.741**     2.0
                    ***077.101**     2.0
                    ***078.571**     2.0
                    ***089.161**     2.0
                    ***105.921**     2.0
                    ***164.701**     2.0
                    ***243.551**     2.0
                    ***353.071**     2.0
                    ***417.841**     2.0
                    ***419.101**     2.0
                    ***443.371**     2.0
                    ***474.991**     2.0
                    ***499.038**     2.0
                    ***510.501**     2.0
                    ***529.020**     2.0
                    ***603.141**     2.0
                    ***669.108**     2.0
                    ***678.531**     2.0
                    ***710.081**     2.0
                    ***874.187**     2.0
                    ***898.211**     2.0
                    ***915.031**     2.0
                    ***930.011**     2.0
Corumbá             ***691.951**     2.0
Costa Rica          ***982.621**     2.0
Coxim               ***297.771**     2.0
Dourados            ***508.051**     2.0
Mundo Novo          ***263.588**     2.0
                    ***950.081**     2.0
Nova Andradina      ***210.691**     2.0
                    ***762.550**     3.0
Sidrolândia         ***088.501**     3.0
Três Lagoas         ***523.201**     2.0